In [1]:
import pandas as pd
import numpy as np
import math
import datetime
from scipy import stats
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

Improvement:
1. Buat monthly rebalancing
2. Monte Carlo random period
3. Tambah parameter dgn sistem ranking (tanpa melalui create_database)

Factors yg didapat so far:
1. Size
2. Value
3. Momentum
4. Illiquidity
5. Price/Profitability

Use Python 3.9.15

In [ ]:
pwd

In [13]:
database_1 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 1.csv')
database_2 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 2.csv')
database_3 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 3.csv')
database_4 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 4.csv')
database_5 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 5.csv')
database_6 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 6.csv')
database_7 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 7.csv')
database_8 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 8.csv')
database_9 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 9.csv')
database_10 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 10.csv')
database_11 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 11.csv')
database_12 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 12.csv')
database_13 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 13.csv')
database_14 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 14.csv')
database_15 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 15.csv')
database_16 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 16.csv')
database_17 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 17.csv')
database_18 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 18.csv')
database_19 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 19.csv')
database_20 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 20.csv')
database_21 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 21.csv')
database_22 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 22.csv')
database_23 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 23.csv')
database_24 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 24.csv')
database_25 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 25.csv')
database_26 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 26.csv')
database_27 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 27.csv')
database_28 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 28.csv')
database_29 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 29.csv')
database_30 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 30.csv')
database = pd.concat([database_1,database_2,database_3,database_4,database_5,database_6,database_7,database_8,database_9,database_10,
                      database_11,database_12,database_13,database_14,database_15,database_16,database_17,database_18,database_19,database_20,
                      database_21,database_22,database_23,database_24,database_25,database_26,database_27,database_28,database_29,database_30])
database = database.drop(columns='Unnamed: 0')
#Substitusi nilai Open, High, Low Price yg 0 dengan Close Price
database['Open Price'] = np.where(database['Open Price'].eq(0),database['Close Price'],database['Open Price'])
database['High Price'] = np.where(database['High Price'].eq(0),database['Close Price'],database['High Price'])
database['Low Price'] = np.where(database['Low Price'].eq(0),database['Close Price'],database['Low Price'])

In [ ]:
database.columns

In [17]:
#BASIC PARAMETER
start_date = '2010-01-01'
end_date = '2025-03-21' #'2025-03-21'
hold_period = 21 #in days
factor = 'Book/Market'
weighting = 'Value-Weight' #'Value-Weight','Equal-Weight'
hold_type = 'Periodic' #'Periodic' or 'First-Day in Month'
partition = 10 #Partition menentukan jumlah portfolio sesuai quartil. Contoh: partition 5 berarti satu portfolio mengandung 20% saham pada satu waktu



In [18]:
#SCREENING PARAMETER
screening_param = ['Market Cap','Total Ekuitas','Close Price','Median Transaction Value 63 Days'] #Params
screening_sign = ['Above','Above','Above','Above'] #Above, Below, Not Equal, or Between. If Between, assign a list in screening_val: screening_val = [[min,max]]
screening_val = [100_000_000_000,0,50,100_000_000] #Numeric variable


In [5]:
#Drop Stocks
drop_list = ['ihsg','excl']

In [19]:
database_date_list = list(database['Date'].sort_values().unique()) 
if hold_type == 'Periodic':
    selected_date_list = database_date_list[::hold_period+1]
elif hold_type == 'First-Day in Month':
    database['Datetime'] = pd.to_datetime(database['Date'])
    database['Year'] = database['Datetime'].dt.year
    database['Month'] = database['Datetime'].dt.month
    selected_date_list_cgpt = database.groupby(['Year', 'Month'])['Datetime'].min().sort_values().dt.strftime('%Y-%m-%d').tolist()
    database = database.drop(columns=['Year', 'Month','Datetime'])
else:
    raise TypeError('hold_type wrong. Select "Periodic" or "First-Day in Month"')
buy_date_list = [d for d in selected_date_list if d >= start_date and d <= end_date]
database_date_array = np.array(database_date_list)
buy_date_array = np.array(buy_date_list[1:]) 
buy_date_index = np.searchsorted(database_date_array, buy_date_array)
sell_date_index = buy_date_index - 1
sell_date_array = database_date_array[sell_date_index]
sell_date_list = sell_date_array.tolist()
param_date_index = [(database_date_list.index(p)-1) for p in buy_date_list]
param_date_list = [database_date_list[p] for p in param_date_index]
basic_param = ['Date','Kode','Market Cap',factor]
basic_param = list(set(basic_param))
extend_param = basic_param.copy()
extend_param.extend(screening_param)
extend_param = list(set(extend_param))
database_part = database[extend_param]  
n_quantile=list(range(partition))
reb_comp_dic = {}
for q in n_quantile:
    reb_comp_dic[f"reb_comp_{q}"] = []
for en,i in enumerate(param_date_list):
    database_i = database_part[database_part['Date']==i]
    if hold_type=='Periodic': 
        pass 
    elif hold_type=='First-Day in Month':
        pass
    else: raise TypeError('hold_type wrong. Select "Periodic" or "First-Day in Month"')
    database_i = database_i.dropna()
    if len(drop_list)>0:
        for r in drop_list:
            database_i = database_i[database_i['Kode']!=r]
    else:
        pass
    if len(screening_param) == len(screening_sign) == len(screening_val):
        for ens,j in enumerate(screening_param):  #Initial Screening
            if screening_sign[ens] == 'Above':
                database_i = database_i[database_i[j]>screening_val[ens]]
            elif screening_sign[ens] == 'Below':
                database_i = database_i[database_i[j]<screening_val[ens]]
            elif screening_sign[ens] == 'Not Equal':
                database_i = database_i[database_i[j]!=screening_val[ens]]
            elif screening_sign[ens] == 'Between':
                database_i = database_i[(database_i[j]>=screening_val[ens][0])&(database_i[j]<=screening_val[ens][1])]
            else:
                raise TypeError('screening_sign wrong. Select either Above or Below')
    else:
        raise Exception('Number of elements in screening parameter list are different')
    database_i = database_i[basic_param]
    database_i['Quintile'] = pd.qcut(database_i[factor], q=partition, labels=False)
    portfolio_dic = {}
    for q in n_quantile:
        portfolio_dic[f"portfolio_{q}"] = database_i[database_i['Quintile']==q]
    for k in portfolio_dic.keys():  
        portfolio_k = portfolio_dic[k]
        if weighting=='Equal-Weight':
            portfolio_k['Weight'] = 1/len(portfolio_k)
        elif weighting=='Value-Weight':
            portfolio_k['Weight'] = portfolio_k['Market Cap']/portfolio_k['Market Cap'].sum()
        else:
            raise TypeError('weighting wrong. See notes in the parameter setting section')
        portfolio_dic[k] = portfolio_k
    for q in n_quantile:
        portfolio_k = portfolio_dic[f"portfolio_{q}"]
        try:
            portfolio_k['From'] = buy_date_list[en]
            portfolio_k['To'] = sell_date_list[en]
            portfolio_k['Number'] = en + 1
            portfolio_dic[f"portfolio_{q}"] = portfolio_k
            reb_comp_dic[f"reb_comp_{q}"].append(portfolio_k)
        except IndexError:
            pass
for q in n_quantile:
    reb_comp_q = pd.concat(reb_comp_dic[f"reb_comp_{q}"])
    reb_comp_q = reb_comp_q[['Kode','Weight','From','To','Number','Quintile']]
    reb_comp_q = reb_comp_q.reset_index(drop=True)
    reb_comp_dic[f"reb_comp_{q}"] = reb_comp_q


CODE REVIEW SEKALI LAGI SEBELUM 

In [ ]:
buy_sell_date_list = buy_date_list + sell_date_list #
database_price = database[['Kode','Date','Open Price','High Price','Low Price','Close Price']] 
database_price = database_price[database_price['Date'].isin(buy_sell_date_list)] #
database_price['Kode + Date'] = database_price['Kode'].astype('str') + ' ' + database_price['Date'].astype('str')
reb_comp_act_dic = {}
for q in n_quantile:
    reb_comp = reb_comp_dic[f'reb_comp_{q}'].sort_values(by=['Number', 'Kode']).reset_index(drop=True)
    database_price_clean = database_price.drop(columns=['Kode', 'Date'])
    grouped = reb_comp.groupby(['Number', 'From', 'To'])['Kode'].apply(list).reset_index()
    exploded = grouped.explode('Kode').reset_index(drop=True)
    buy_df = exploded[['Kode', 'From']].rename(columns={'From': 'Date'})
    buy_df['Date'] = buy_df['Date'].astype(str)
    buy_df['Action'] = 'BUY'
    buy_df['Kode + Date'] = buy_df['Kode'].astype(str) + ' ' + buy_df['Date']
    sell_df = exploded[['Kode', 'To']].rename(columns={'To': 'Date'})
    sell_df['Date'] = sell_df['Date'].astype(str)
    sell_df['Action'] = 'SELL'
    sell_df['Kode + Date'] = sell_df['Kode'].astype(str) + ' ' + sell_df['Date']
    buy_df = buy_df.merge(database_price_clean, how='left', on='Kode + Date').drop(columns='Kode + Date')
    sell_df = sell_df.merge(database_price_clean, how='left', on='Kode + Date').drop(columns='Kode + Date')
    buy_df = buy_df.sort_values(['Date', 'Kode']).reset_index(drop=True)
    sell_df = sell_df.sort_values(['Date', 'Kode']).reset_index(drop=True)
    reb_comp_act_dic[f"reb_comp_act_{q}"] = []
    for num in grouped['Number']:
        buy_slice = buy_df[exploded['Number'] == num].reset_index(drop=True)
        sell_slice = sell_df[exploded['Number'] == num].reset_index(drop=True)
        if buy_slice['Kode'].duplicated().any():
            print("Stock list is not unique")
            break
        reb_comp_act_dic[f"reb_comp_act_{q}"].append(buy_slice)
        reb_comp_act_dic[f"reb_comp_act_{q}"].append(sell_slice)
for q in n_quantile:
    reb_comp_act_q = pd.concat(reb_comp_act_dic[f"reb_comp_act_{q}"])
    reb_comp_act_q = reb_comp_act_q.reset_index(drop=True)
    reb_comp_act_dic[f"reb_comp_act_{q}"] = reb_comp_act_q
database_price = database_price.drop(columns='Kode + Date')

In [21]:
transaction_dic = {}
for q in n_quantile:
    transaction_dic[f"transaction_{q}"] = []
for q in n_quantile:
    reb_comp_act = reb_comp_act_dic[f"reb_comp_act_{q}"]
    reb_comp = reb_comp_dic[f'reb_comp_{q}']
    day_number = 0
    reb_comp_act_full_day_list = []
    for i in list(reb_comp_act['Date'].sort_values().unique()):
        reb_comp_act_i = reb_comp_act[reb_comp_act['Date']==i]
        reb_comp_act_i['Day Number'] = day_number
        reb_comp_i = reb_comp[reb_comp['From']==i]
        reb_comp_i = reb_comp_i.rename(columns={'From':'Date'})
        reb_comp_i['Kode + Date'] = reb_comp_i['Kode'].astype('str') + ' ' + reb_comp_i['Date'].astype('str') 
        reb_comp_act_i_buy_list = []
        reb_comp_act_i['Initial Capital'] = 1_000_000_000
        reb_comp_act_i['Reference Price'] = reb_comp_act_i['Close Price']
        reb_comp_act_i['Price/Lot'] = reb_comp_act_i['Reference Price'].astype('float64')*100
        reb_comp_act_i['Kode + Date'] = reb_comp_act_i['Kode'].astype('str') + ' ' + reb_comp_act_i['Date'].astype('str') 
        if reb_comp_act_i['Action'].values[0] == 'BUY':
            if day_number == 0:
                reb_comp_act_i['Reference Capital'] = 1_000_000_000
            else:
                reb_comp_act_i['Reference Capital'] = reb_comp_act_prev['Total Capital'].iloc[-1]
            reb_comp_act_i = reb_comp_act_i.merge(reb_comp_i[['Kode + Date','Weight']],on='Kode + Date',how='left')
            reb_comp_act_i['Buy Price'] = reb_comp_act_i['Price/Lot']
            reb_comp_act_i['Sell Price'] = np.nan
            reb_comp_act_i['Buy Lot'] = reb_comp_act_i['Reference Capital']*reb_comp_act_i['Weight']/reb_comp_act_i['Buy Price']
            reb_comp_act_i['Sell Lot'] = np.nan
            reb_comp_act_i['Hold Lot'] = np.nan
            reb_comp_act_i['Capital Allocation'] = reb_comp_act_i['Reference Capital']*reb_comp_act_i['Weight']
            reb_comp_act_i['Invested'] = reb_comp_act_i['Buy Price']*reb_comp_act_i['Buy Lot']
            reb_comp_act_i['Sold'] = np.nan
            reb_comp_act_i['Market Value'] = reb_comp_act_i['Close Price']*reb_comp_act_i['Buy Lot']*100
            reb_comp_act_i['Potential P&L'] = reb_comp_act_i['Market Value'] - reb_comp_act_i['Invested']
            reb_comp_act_i['Realized P&L'] = np.nan
            reb_comp_act_i['Cash'] = reb_comp_act_i['Capital Allocation'] - reb_comp_act_i['Invested']
            reb_comp_act_i['ROI'] = np.nan
        elif reb_comp_act_i['Action'].values[0] == 'SELL':
            reb_comp_act_i['Reference Capital'] = reb_comp_act_prev['Total Market Value'].iloc[-1]
            reb_comp_act_i['Reference Price'] = reb_comp_act_i['Close Price']
            reb_comp_act_i['Weight'] = 0
            reb_comp_act_i['Buy Price'] = np.nan
            reb_comp_act_i['Sell Price'] = reb_comp_act_i['Price/Lot']
            reb_comp_act_i = reb_comp_act_i.merge(reb_comp_act_prev[['Kode','Buy Lot']],on='Kode',how='left')
            reb_comp_act_i = reb_comp_act_i.rename(columns={'Buy Lot':'Sell Lot'})
            reb_comp_act_i['Buy Lot'] = np.nan
            reb_comp_act_i['Hold Lot'] = np.nan
            reb_comp_act_i['Capital Allocation'] = np.nan
            reb_comp_act_i['Invested'] = reb_comp_act_prev['Invested']
            reb_comp_act_i['Sold'] = reb_comp_act_i['Sell Price']*reb_comp_act_i['Sell Lot']
            reb_comp_act_i['Market Value'] = np.nan
            reb_comp_act_i['Potential P&L'] = np.nan
            reb_comp_act_i['Realized P&L'] = reb_comp_act_i['Sold']-reb_comp_act_i['Invested']
            reb_comp_act_i['Cash'] = reb_comp_act_prev['Cash'] + reb_comp_act_i['Sold']
            reb_comp_act_i['ROI'] = (reb_comp_act_i['Sold']-reb_comp_act_i['Invested'])/reb_comp_act_i['Invested']
        else: pass
        reb_comp_act_i['Total Invested'] = reb_comp_act_i['Invested'].sum()
        reb_comp_act_i['Total Market Value'] = reb_comp_act_i['Market Value'].sum()
        reb_comp_act_i['Total Potential P&L'] = reb_comp_act_i['Potential P&L'].sum()
        reb_comp_act_i['Total Realized P&L'] =reb_comp_act_i['Realized P&L'].sum()
        reb_comp_act_i['Total Cash'] =reb_comp_act_i['Cash'].sum() 
        reb_comp_act_i['Total Capital'] = reb_comp_act_i['Total Market Value'] + reb_comp_act_i['Total Cash']
        reb_comp_act_i['Weight'] = reb_comp_act_i['Weight'].fillna(value=reb_comp_act_i['Market Value']/reb_comp_act_i['Total Market Value'])
        day_number = day_number + 1
        transaction_dic[f"transaction_{q}"].append(reb_comp_act_i)
        reb_comp_act_prev = reb_comp_act_i.copy()
for q in n_quantile:
    transaction_q = pd.concat(transaction_dic[f"transaction_{q}"])
    transaction_q = transaction_q.reset_index(drop=True)
    transaction_dic[f"transaction_{q}"] = transaction_q

In [22]:
hist_capital_dic = {}
for q in n_quantile:
    hist_capital_dic[f"hist_capital_{q}"] = []
for q in n_quantile:
    transaction_q = transaction_dic[f"transaction_{q}"]
    transaction_buy = transaction_q[transaction_q['Action']=='BUY']
    transaction_buy = transaction_buy.sort_values(by='Date')
    for i in transaction_buy['Date'].unique():
        transaction_buy_i = transaction_buy[transaction_buy['Date']==i]
        transaction_buy_i = transaction_buy_i[['Date','Total Capital']]
        transaction_buy_i = transaction_buy_i.head(1)
        hist_capital_dic[f"hist_capital_{q}"].append(transaction_buy_i)
for q in n_quantile:
    hist_capital_q = pd.concat(hist_capital_dic[f"hist_capital_{q}"])
    hist_capital_q['Return'] = hist_capital_q['Total Capital'].pct_change()
    hist_capital_q = hist_capital_q.reset_index(drop=True)
    hist_capital_dic[f"hist_capital_{q}"] = hist_capital_q
summary_dic = {}
for q in n_quantile:
    hist_capital_q = hist_capital_dic[f"hist_capital_{q}"]
    hist_capital_q = hist_capital_q.dropna()
    summary_dic[f"Average Portfolio Return Q{q}"] = hist_capital_q['Return'].mean()
first_q = n_quantile[0]
last_q = n_quantile[-1]
t_stat, p_value = stats.ttest_ind(hist_capital_dic[f"hist_capital_{first_q}"]['Return'].dropna(),hist_capital_dic[f"hist_capital_{last_q}"]['Return'].dropna(), equal_var=False)
summary_dic[f'Average Portfolio Return Top-Bottom'] = abs(summary_dic[f"Average Portfolio Return Q{first_q}"] - summary_dic[f"Average Portfolio Return Q{last_q}"])
summary_dic['t-stat'] = t_stat
summary_dic['p-value'] = p_value
ihsg = database[['Date','IHSG']]
ihsg = ihsg.drop_duplicates(subset='Date')
ihsg_ret = hist_capital_q[['Date']].merge(ihsg,on='Date',how='left')
ihsg_ret['Return'] = ihsg_ret['IHSG'].pct_change()
ihsg_ret = ihsg_ret.dropna()
summary_dic['Average Market Return'] = ihsg_ret['Return'].mean()
factor = factor.replace('/','-to-')
results = pd.DataFrame([summary_dic])
#results.to_excel(f'Factor Testing Results for {factor} with {weighting} methodology and {hold_type} rebalancing.xlsx')



In [ ]:
results

,Average Portfolio Return Q0,Average Portfolio Return Q1,Average Portfolio Return Q2,Average Portfolio Return Q3,Average Portfolio Return Q4,Average Portfolio Return Q5,Average Portfolio Return Q6,Average Portfolio Return Q7,Average Portfolio Return Q8,Average Portfolio Return Q9,Average Portfolio Return Top-Bottom,t-stat,p-value,Average Market Return
0,-0.002059,0.004559,0.003131,0.00653,0.007025,0.006703,0.008771,0.009551,0.010923,0.013359,0.015418,-1.851923,0.064963,0.00648
